In [39]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.utils import to_categorical

In [40]:
(previsores_treinamento, classe_treinamento), (previsores_teste, classe_teste) = mnist.load_data()

In [41]:
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_teste = previsores_teste.astype('float32') / 255

classe_dummy_treinamento = to_categorical(classe_treinamento)
classe_dummy_teste = to_categorical(classe_teste)

In [42]:
previsores_treinamento = previsores_treinamento.reshape((len(previsores_treinamento), np.prod(previsores_treinamento.shape[1:])))
previsores_teste = previsores_teste.reshape((len(previsores_teste), np.prod(previsores_teste.shape[1:])))

In [43]:
autoencoder = Sequential()
autoencoder.add(Dense(units = 32, activation = 'relu', input_dim = 784))
autoencoder.add(Dense(units = 784, activation = 'sigmoid'))
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                    metrics = ['accuracy'])

In [44]:
autoencoder.fit(previsores_treinamento, previsores_treinamento,
                epochs = 5, batch_size = 256, validation_data = (previsores_teste, previsores_teste))

Epoch 1/5
235/235 [==============================] - 2s 6ms/step - loss: 0.2745 - accuracy: 0.0104 - val_loss: 0.1866 - val_accuracy: 0.0113
Epoch 2/5
235/235 [==============================] - 1s 6ms/step - loss: 0.1692 - accuracy: 0.0119 - val_loss: 0.1526 - val_accuracy: 0.0127
Epoch 3/5
235/235 [==============================] - 1s 5ms/step - loss: 0.1435 - accuracy: 0.0111 - val_loss: 0.1330 - val_accuracy: 0.0096
Epoch 4/5
235/235 [==============================] - 1s 5ms/step - loss: 0.1279 - accuracy: 0.0105 - val_loss: 0.1208 - val_accuracy: 0.0095
Epoch 5/5
235/235 [==============================] - 1s 5ms/step - loss: 0.1178 - accuracy: 0.0106 - val_loss: 0.1126 - val_accuracy: 0.0092


In [45]:
dimensao_original = Input(shape=(784,))
camada_encoder = autoencoder.layers[0]
encoder = Model(dimensao_original, camada_encoder(dimensao_original))

In [46]:
previsores_treinamento_codificados = encoder.predict(previsores_treinamento)
previsores_teste_codificados = encoder.predict(previsores_teste)

313/313 [==============================] - 0s 1ms/step


In [47]:
c1 = Sequential()
c1.add(Dense(units = 397, activation = 'relu', input_dim = 784))
c1.add(Dense(units = 397, activation = 'relu'))
c1.add(Dense(units = 10, activation = 'softmax'))
c1.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
            metrics = ['accuracy'])
c1.fit(previsores_treinamento, classe_dummy_treinamento,
        epochs = 10, batch_size = 256, validation_data = (previsores_teste, classe_dummy_teste))

In [ ]:
c2 = Sequential()
c2.add(Dense(units = 21, activation = 'relu', input_dim = 32))
c2.add(Dense(units = 21, activation = 'relu'))
c2.add(Dense(units = 10, activation = 'softmax'))
c2.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
            metrics = ['accuracy'])
c2.fit(previsores_treinamento_codificados, classe_dummy_treinamento,
        epochs = 10, batch_size = 256, validation_data = (previsores_teste_codificados, classe_dummy_teste))